## Deliverable 2. Create a Customer Travel Destinations Map.

In [22]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Jacksonville,US,30.33,-81.66,63.00,58,1,11.41,clear sky
1,1,Ushuaia,AR,-54.80,-68.30,50.00,61,75,29.97,broken clouds
2,2,Arraial Do Cabo,BR,-22.97,-42.03,78.80,83,75,11.41,broken clouds
3,3,Bonavista,CA,48.65,-53.11,36.00,58,48,17.00,scattered clouds
4,4,Mana,GF,5.66,-53.78,87.55,65,65,13.85,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Arraial Do Cabo,BR,-22.97,-42.03,78.80,83,75,11.41,broken clouds
4,4,Mana,GF,5.66,-53.78,87.55,65,65,13.85,broken clouds
7,7,Vaini,TO,-21.20,-175.20,71.60,78,75,8.05,broken clouds
10,10,Gizo,SB,-8.10,156.84,80.19,82,100,11.59,moderate rain
15,15,Rikitea,PF,-23.12,-134.97,74.97,71,64,5.79,broken clouds
20,20,Port-Gentil,GA,-0.72,8.78,84.20,79,20,12.75,few clouds
28,28,San Cosme Y Damian,PY,-27.32,-56.35,71.60,53,0,9.17,clear sky
31,31,Castro Verde,PT,37.70,-8.09,77.00,36,0,2.24,clear sky
33,33,Taveta,KE,-3.40,37.67,75.20,64,75,11.41,broken clouds
37,37,Caravelas,BR,-17.71,-39.25,84.18,60,8,12.46,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                205
City                   205
Country                202
Lat                    205
Lng                    205
Max Temp               205
Humidity               205
Cloudiness             205
Wind Speed             205
Current Description    205
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)].dropna()
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Arraial Do Cabo,BR,-22.97,-42.03,78.80,83,75,11.41,broken clouds
4,4,Mana,GF,5.66,-53.78,87.55,65,65,13.85,broken clouds
7,7,Vaini,TO,-21.20,-175.20,71.60,78,75,8.05,broken clouds
10,10,Gizo,SB,-8.10,156.84,80.19,82,100,11.59,moderate rain
15,15,Rikitea,PF,-23.12,-134.97,74.97,71,64,5.79,broken clouds
20,20,Port-Gentil,GA,-0.72,8.78,84.20,79,20,12.75,few clouds
28,28,San Cosme Y Damian,PY,-27.32,-56.35,71.60,53,0,9.17,clear sky
31,31,Castro Verde,PT,37.70,-8.09,77.00,36,0,2.24,clear sky
33,33,Taveta,KE,-3.40,37.67,75.20,64,75,11.41,broken clouds
37,37,Caravelas,BR,-17.71,-39.25,84.18,60,8,12.46,clear sky


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Arraial Do Cabo,BR,78.80,broken clouds,-22.97,-42.03,
4,Mana,GF,87.55,broken clouds,5.66,-53.78,
7,Vaini,TO,71.60,broken clouds,-21.20,-175.20,
10,Gizo,SB,80.19,moderate rain,-8.10,156.84,
15,Rikitea,PF,74.97,broken clouds,-23.12,-134.97,
20,Port-Gentil,GA,84.20,few clouds,-0.72,8.78,
28,San Cosme Y Damian,PY,71.60,clear sky,-27.32,-56.35,
31,Castro Verde,PT,77.00,clear sky,37.70,-8.09,
33,Taveta,KE,75.20,broken clouds,-3.40,37.67,
37,Caravelas,BR,84.18,clear sky,-17.71,-39.25,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

print("Beginning Data Retrieval     ")
print("-----------------------------")

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except IndexError:
         print("Hotel not found... skipping.")
            
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")        

Beginning Data Retrieval     
-----------------------------
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
-----------------------------
Data Retrieval Complete      
-----------------------------


In [37]:
# 7. Drop the rows where there is no Hotel Name
hotel_df = hotel_df.replace('',np.nan)
hotel_df = hotel_df.dropna()
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Arraial Do Cabo,BR,78.80,broken clouds,-22.97,-42.03,Pousada Porto Praia
4,Mana,GF,87.55,broken clouds,5.66,-53.78,LE SAMANA
7,Vaini,TO,71.60,broken clouds,-21.20,-175.20,Keleti Beach Resort
10,Gizo,SB,80.19,moderate rain,-8.10,156.84,Imagination Island
15,Rikitea,PF,74.97,broken clouds,-23.12,-134.97,Pension Maro'i
...,...,...,...,...,...,...,...
502,Sao Joao Da Ponte,BR,74.71,overcast clouds,-15.93,-44.01,Hotel da Cidade
512,San Policarpo,PH,82.15,overcast clouds,12.18,125.51,Zal-Ann Beach Resort
520,Iracoubo,GF,86.72,overcast clouds,5.48,-53.20,Le Gite D Iracoubo
523,Kailua,US,75.20,clear sky,21.40,-157.74,Sheffield House Bed & Breakfast


In [38]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [39]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl><dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp}  °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [40]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Call the figure to plot the data.
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))